# Quick Model Merging Experiment

This notebook demonstrates how to perform a quick model merging experiment using the `MergingUriel` codebase. 
We will:
1.  Select a target language and source models.
2.  Compute similarity weights (the "smarter way").
3.  Merge the models.
4.  Evaluate the merged model on the MASSIVE dataset.

In [ ]:
import os
import sys
import pandas as pd
import torch
from pathlib import Path
from datetime import datetime

# Setup paths to ensure imports work correctly
project_root = os.path.abspath(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

submodule_path = os.path.join(project_root, 'submodules/auto_merge_llm')
if submodule_path not in sys.path:
    sys.path.insert(0, submodule_path)

# Import merginguriel components
from merginguriel.run_merging_pipeline_refactored import (
    MergeConfig, 
    WeightCalculatorFactory, 
    ModelMerger, 
    OutputManager, 
    Evaluator
)
from merginguriel.evaluate_specific_model import evaluate_specific_model, save_evaluation_results, create_results_folder
from merginguriel.naming_config import naming_manager

## 1. Configuration

Set your experiment parameters here. 
You can choose the target language (locale), the similarity type ('URIEL' or 'REAL'), and the merge mode.

In [ ]:
# --- Experiment Parameters ---
TARGET_LANG = "sq-AL"  # Example: Albanian
MERGE_MODE = "similarity" # Options: 'similarity', 'average', 'fisher', 'ties', 'task_arithmetic', 'slerp'
SIMILARITY_TYPE = "URIEL" # Options: 'URIEL' (linguistic) or 'REAL' (empirical)
NUM_LANGUAGES = 5      # Number of top-k similar languages to merge
BASE_MODEL_TYPE = "xlm-roberta-base" # or 'xlm-roberta-large'
INCLUDE_TARGET = False # Set to True to include the target language model itself in the merge (IT mode)

# Ensure models directory exists
MODELS_ROOT = "haryos_model" if BASE_MODEL_TYPE == "xlm-roberta-base" else "haryos_model_large"
print(f"Using models from: {MODELS_ROOT}")

## 2. Calculate Weights & Select Models

We use the `WeightCalculator` to automatically find the most similar languages and calculate their mixing weights.

In [ ]:
# Create Merge Configuration
config = MergeConfig(
    mode=MERGE_MODE,
    target_lang=TARGET_LANG,
    base_model=BASE_MODEL_TYPE,
    num_languages=NUM_LANGUAGES,
    similarity_type=SIMILARITY_TYPE,
    include_target=INCLUDE_TARGET,
    base_model_dir=os.path.join(project_root, MODELS_ROOT),
    # Additional params for advanced methods
    similarity_source="dense", # Use dense for on-the-fly calculation if needed
    top_k=20,
    sinkhorn_iters=20
)

# Initialize Weight Calculator
calculator = WeightCalculatorFactory.create_calculator(MERGE_MODE)

# Calculate Weights
try:
    models_and_weights, base_model_info = calculator.calculate_weights(config)
    print("\nSelected Models and Weights:")
    print(f"Base Model: {base_model_info.model_name} (Weight: {base_model_info.weight:.4f})")
    for path, info in models_and_weights.items():
        print(f" - {info.locale}: {info.weight:.4f} ({path})")
except Exception as e:
    print(f"Error calculating weights: {e}")
    # Fallback or exit logic here

## 3. Perform Merging

Now we merge the selected models using the specified strategy.

In [ ]:
# Initialize Model Merger
merger = ModelMerger(config)

# Perform Merge
print(f"Merging models using {MERGE_MODE} strategy...")
merged_model, tokenizer = merger.merge_models(models_and_weights, base_model_info)
print("Merge complete!")

## 4. Save Merged Model

Save the merged model to the `merged_models` directory.

In [ ]:
# Initialize Output Manager
output_manager = OutputManager(project_root, merged_models_dir="merged_models")

# Save Model
output_dir = output_manager.save_model_and_details(
    merged_model, 
    tokenizer, 
    config, 
    models_and_weights, 
    base_model_info
)

print(f"Merged model saved to: {output_dir}")

## 5. Evaluation

Evaluate the merged model on the MASSIVE dataset for the target locale.

In [ ]:
print(f"Evaluating merged model on {TARGET_LANG}...")

# Create results directory
results_folder = create_results_folder(
    base_model=output_dir, 
    locale=TARGET_LANG, 
    prefix=f"{MERGE_MODE}_{NUM_LANGUAGES}lang_{SIMILARITY_TYPE}")

# Run Evaluation
results = evaluate_specific_model(
    model_name=output_dir, 
    locale=TARGET_LANG, 
    eval_folder=results_folder
)

if results:
    save_evaluation_results(results, results_folder)
    print(f"\nAccuracy: {results['performance']['accuracy']:.4f}")
    print(f"Results saved to: {results_folder}")
else:
    print("Evaluation failed.")

## 6. Cleanup (Optional)

Remove the merged model to save space if you only care about the results.

In [ ]:
# import shutil
# shutil.rmtree(output_dir)
# print(f"Cleaned up {output_dir}")

## 7. Manual Verification (No Library Imports)

The following cells perform the exact same steps (Calculate Weights -> Merge -> Evaluate) but implement the logic manually within the notebook cells.

This ensures that the library functions are working as expected.

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from datasets import load_dataset
from tqdm.auto import tqdm

# Helper functions for manual weight calculation
def manual_sinkhorn_normalize(matrix, iterations=20, epsilon=1e-9):
    result = matrix.copy()
    for _ in range(iterations):
        # Normalize rows
        result /= (result.sum(axis=1, keepdims=True) + epsilon)
        # Normalize cols
        result /= (result.sum(axis=0, keepdims=True) + epsilon)
    return result

def manual_filter_top_k(matrix, k):
    sparse_matrix = matrix.copy()
    # Zero out diagonal (self-similarity) based on library implementation
    np.fill_diagonal(sparse_matrix, 0)
    
    num_rows = sparse_matrix.shape[0]
    for i in range(num_rows):
        row = sparse_matrix[i, :]
        if len(row) > k:
            # Find k-th largest value
            kth_largest = np.partition(row, -k)[-k]
            # Zero out anything smaller than it
            row[row < kth_largest] = 0
    return sparse_matrix

In [ ]:
# 1. Manual Weight Calculation
print("--- Manual Weight Calculation ---")

# Load Matrix
sim_matrix_path = "language_similarity_matrix_unified.csv"
df = pd.read_csv(sim_matrix_path, index_col=0)

# Deduplicate
df = df[~df.index.duplicated(keep='first')]
df = df.loc[:, ~df.columns.duplicated(keep='first')]

# Select Target
manual_target_locale = TARGET_LANG # Use same var from above
if manual_target_locale not in df.index:
    raise ValueError(f"{manual_target_locale} not found in matrix")

# Process Matrix
matrix_val = df.values
sparse_matrix = manual_filter_top_k(matrix_val, k=20)
normalized_matrix = manual_sinkhorn_normalize(sparse_matrix, iterations=20)

# Reconstruct DataFrame
processed_df = pd.DataFrame(normalized_matrix, index=df.index, columns=df.columns)
target_weights = processed_df.loc[manual_target_locale]

# Get Top-K Weights (descending)
top_n_weights = target_weights[target_weights > 0].sort_values(ascending=False)

# Filter for num_languages (excluding target itself if ET mode)
if not INCLUDE_TARGET:
    top_n_weights = top_n_weights[top_n_weights.index != manual_target_locale]

top_n_weights = top_n_weights[:NUM_LANGUAGES]

# Normalize to sum to 1.0 for linear merge
total_weight = top_n_weights.sum()
final_weights = top_n_weights / total_weight

print(f"Manual Weights for {manual_target_locale}:")
print(final_weights)

In [ ]:
# 2. Manual Merging
print("\n--- Manual Merging ---")

base_model_name = BASE_MODEL_TYPE
models_root_dir = MODELS_ROOT

# Identify the first model to use as the architecture base
first_locale = final_weights.index[0]
first_model_path = os.path.join(project_root, models_root_dir, f"{base_model_name}_massive_k_{first_locale}")

print(f"Initializing base architecture from: {first_model_path}")
config = AutoConfig.from_pretrained(first_model_path)
manual_merged_model = AutoModelForSequenceClassification.from_config(config)
merged_state_dict = manual_merged_model.state_dict()

# Reset to zero
for key in merged_state_dict:
    if merged_state_dict[key].dtype.is_floating_point:
        merged_state_dict[key].zero_()

# Weighted Sum
for locale, weight in final_weights.items():
    model_path = os.path.join(project_root, models_root_dir, f"{base_model_name}_massive_k_{locale}")
    print(f"Loading {model_path} (w={weight:.4f})")
    
    try:
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
    except OSError:
        print(f"Warning: Could not load {model_path}, skipping...")
        continue
        
    sd = model.state_dict()
    for key in merged_state_dict:
        if key in sd and sd[key].dtype.is_floating_point:
            merged_state_dict[key] += sd[key] * weight

# Load merged weights back
manual_merged_model.load_state_dict(merged_state_dict)
print("Manual Merge Complete.")

In [ ]:
# 3. Manual Evaluation
print(f"\n--- Manual Evaluation on {manual_target_locale} ---")

dataset = load_dataset("AmazonScience/massive", manual_target_locale, split="test")
tokenizer = AutoTokenizer.from_pretrained(first_model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
manual_merged_model.to(device)
manual_merged_model.eval()

correct = 0
total = 0

for example in tqdm(dataset, desc="Evaluating"):
    inputs = tokenizer(example['utt'], return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = manual_merged_model(**inputs)
        pred_idx = outputs.logits.argmax(dim=-1).item()
        
    # MASSIVE uses integer intent labels
    if pred_idx == example['intent']:
        correct += 1
    total += 1

accuracy = correct / total
print(f"\nManual Accuracy: {accuracy:.4f}")

# 8. Angle Between Weights

In [1]:
from glob import glob

haryos_model = glob("haryos_model/*")
haryos_model_large = glob("haryos_model_large/*")
list_locale = [path.split("_")[-1] for path in haryos_model]
print(f"{list_locale = }")

list_locale = ['mn-MN', 'sq-AL', 'vi-VN', 'id-ID', 'th-TH', 'ka-GE', 'pt-PT', 'de-DE', 'my-MM', 'af-ZA', 'hu-HU', 'hi-IN', 'kn-IN', 'en-US', 'pl-PL', 'it-IT', 'ro-RO', 'ru-RU', 'is-IS', 'ur-PK', 'zh-TW', 'ta-IN', 'sl-SL', 'da-DK', 'fr-FR', 'es-ES', 'am-ET', 'ar-SA', 'fi-FI', 'jv-ID', 'hy-AM', 'ml-IN', 'ko-KR', 'nl-NL', 'tr-TR', 'lv-LV', 'tl-PH', 'el-GR', 'az-AZ', 'fa-IR', 'km-KH', 'sw-KE', 'te-IN', 'ms-MY', 'ja-JP', 'nb-NO', 'cy-GB', 'ca-ES', 'bn-BD']


In [2]:
import torch

# arccos(0) in degrees
torch.rad2deg(torch.acos(torch.tensor(0.0)))

tensor(90.)

In [3]:
# Shape of weight = (64, 64)

dim = 64
example_tensor_1 = torch.randn((dim, dim))
example_tensor_2 = torch.randn((dim, dim))

def angle(weight_1: torch.Tensor, weight_2: torch.Tensor) -> torch.Tensor:
    cos_sim = torch.nn.functional.cosine_similarity(
        weight_1.view(-1),
        weight_2.view(-1),
        dim=0
    )
    angle_rad = torch.acos(torch.clamp(cos_sim, -1.0, 1.0))
    degree = torch.rad2deg(angle_rad)
    return degree, cos_sim

print(angle(example_tensor_1, example_tensor_1))

(tensor(0.), tensor(1.0000))


In [ ]:
import transformers
from transformers import AutoModelForSequenceClassification

finetuned_model_1 = AutoModelForSequenceClassification.from_pretrained(
    haryos_model[0]
)
finetuned_model_1

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [7]:
from transformers import XLMRobertaModel

base_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
base_model

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine